## Linking without deduplication

A simple record linkage model using the `link_only` [link type](https://moj-analytical-services.github.io/splink/settings_dict_guide.html#link_type).

In [3]:
from splink.datasets import splink_datasets
df = splink_datasets.fake_1000

# Split a simple dataset into two, separate datasets which can be linked together.
df_l = df.sample(frac=0.5)
df_r = df.drop(df_l.index)

df_l.head(2)

,unique_id,first_name,surname,dob,city,email,cluster
511,511,Theo,Whhite,2011-05-06,NaN,t.white@west-waller.net,127
561,561,Jones,Noah,1988-02-14,Telford,noah.ones66@smithjncom,137


In [4]:
from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl


settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name",
        "l.surname = r.surname",
    ],
    "comparisons": [
        ctl.name_comparison("first_name",),
        ctl.name_comparison("surname"),
        ctl.date_comparison("dob", cast_strings_to_date=True),
        cl.exact_match("city", term_frequency_adjustments=True),
        ctl.email_comparison("email"),
    ],       
}

In [5]:
linker = DuckDBLinker([df_l, df_r], settings, input_table_aliases=["df_left", "df_right"])
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)


Probability two random records match is estimated to be  0.00333.
This means that amongst all possible pairwise record comparisons, one in 300.17 are expected to match.  With 250,000 total possible comparisons, we expect a total of around 832.86 matching pairs


In [6]:
linker.estimate_u_using_random_sampling(max_pairs=1e6, seed=1)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [7]:
session_dob = linker.estimate_parameters_using_expectation_maximisation("l.dob = r.dob")
session_email = linker.estimate_parameters_using_expectation_maximisation("l.email = r.email")
session_first_name = linker.estimate_parameters_using_expectation_maximisation("l.first_name = r.first_name")


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.dob = r.dob

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob


Level Jaro_winkler_similarity Username >= 0.88 on comparison email not observed in dataset, unable to train m value
Iteration 1: Largest change in params was -0.426 in the m_probability of surname, level `Exact match surname`

Level Jaro_winkler_similarity Username >= 0.88 on comparison email not observed in dataset, unable to train m value
Iteration 2: Largest change in params was 0.124 in probability_two_random_records_match

Level Jaro_winkler_similarity Username >= 0.88 on comparison email not observed in dataset, unable to train m value
Iteration 3: Largest change in params was 0.0405 in probability_two_random_records_match



In [8]:
results = linker.predict(threshold_match_probability=0.9)

In [9]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,surname_l,...,dob_l,dob_r,gamma_dob,city_l,city_r,gamma_city,email_l,email_r,gamma_email,match_key
0,17.811694,0.999996,df_left,df_right,166,168,Elizabeth,Elizabeth,4,Wright,...,1994-02-03,1994-02-01,4,Swansea,Swasea,0,elizabeth.w@sullivan-anderson.info,elizabeth.w@sullivan-anderson.info,4,0
1,19.583788,0.999999,df_left,df_right,994,995,Emma,Emma,4,Lloyd,...,2018-04-17,2018-04-17,5,London,nLonon,0,emmalloyd@jimenez.com,emmalloyd@jimenez.com,4,0
2,10.907431,0.999480,df_left,df_right,256,254,Sofia,Sofia,4,Russell,...,2014-09-03,2015-09-03,4,London,NaN,-1,NaN,sofiarussell9@revira.com,-1,0
3,18.638152,0.999998,df_left,df_right,84,81,Ryan,Ryan,4,Cole,...,1988-05-27,1987-05-27,4,Bristol,NaN,-1,r.cole1@ramtrez-anihony.com,r.cole1@ramirez-anthony.com,3,0
4,9.994794,0.999021,df_left,df_right,448,450,Aisha,Aisha,4,BryBant,...,1993-08-08,1993-07-10,3,Bradfrod,Brddfora,0,aishab64@obrien-flores.com,aishab64@obrien-flores.com,4,0
